#### previous exercise

In [1]:
import pandas

# read the 311 CSV
df_311 = pandas.read_csv("../Chapter05/311_cases.csv")

# drop the DELETE columns
drop_cols = [c for c in df_311.columns if "DELETE" in c]
df_311 = df_311.drop(columns = drop_cols)

# exclude any records with invalid Latitude/Longitude
df_311 = df_311[df_311['Latitude'] > 0]

# convert Opened/Closed to datetime
df_311['Opened'] = pandas.to_datetime(df_311['Opened'])
df_311['Closed'] = pandas.to_datetime(df_311['Closed'])

# subtract the Opened time from the Closed time to get the OpenTime duration
df_311['OpenTime'] = df_311['Closed'] - df_311['Opened']

# create a DataFrame summarizing by neighborhood
df_neighborhood = df_311.groupby("Neighborhood").agg(
    {
        "OpenTime": "mean",
        "CaseID": "count"
    }
)

C:\Users\dav11274\AppData\Local\Temp\1\ipykernel_43580\1460595380.py:14: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_311['Opened'] = pandas.to_datetime(df_311['Opened'])


In [2]:
df_neighborhood

,OpenTime,CaseID
Neighborhood,,
Alamo Square,3 days 04:03:59.753424657,252
Anza Vista,4 days 15:34:21.985714285,76
Apparel City,1 days 22:10:47.308823529,72
Aquatic Park / Ft. Mason,0 days 23:16:31.140740740,143
Ashbury Heights,2 days 07:45:51.923076923,92
...,...,...
West Portal,1 days 19:05:03.544776119,289
Western Addition,2 days 23:59:47.578125,696
Westwood Highlands,6 days 02:19:14.714285714,16


#### mapping the neighborhoods

In [3]:
df_neighborhood['OpenTime'] = df_neighborhood['OpenTime'].dt.days

In [4]:
df_neighborhood = df_neighborhood.reset_index()

In [5]:
import arcgis

sedf_neighborhoods = pandas.DataFrame.spatial.from_featureclass(
    "./Tutorial_06_02.gdb/SF_Find_Neighborhoods"
)

In [6]:
sedf_neighborhoods.head()

,OBJECTID,name,SHAPE
0,1,Seacliff,"{""rings"": [[[-13635909.0659, 4548889.167499997..."
1,2,Lake Street,"{""rings"": [[[-13635207.2463, 4548926.810999997..."
2,3,Presidio National Park,"{""rings"": [[[-13634141.858199999, 4552759.7788..."
3,4,Presidio Terrace,"{""rings"": [[[-13633566.376400001, 4549428.4135..."
4,5,Inner Richmond,"{""rings"": [[[-13633590.3391, 4549283.085600004..."


In [7]:
sedf_merge = sedf_neighborhoods.merge(df_neighborhood, 
                                      how = 'left', 
                                      left_on = 'name', 
                                      right_on = 'Neighborhood'
                                      )

In [8]:
gis = arcgis.GIS()
neighborhood_map = gis.map("San Francisco, CA")
neighborhood_map

MapView(layout=Layout(height='400px', width='100%'))

In [10]:
sedf_merge.spatial.plot(
                colors='coolwarm',
                class_count=20,
                map_widget=neighborhood_map,
                renderer_type='c',
                col='OpenTime'
)

True

In [11]:
neighborhood_fc = sedf_merge.spatial.to_featureclass(
    "./Tutorial_06_02.gdb/case_open_time_by_neighborhood"
)

In [13]:
neighborhood_fc

'C:\\Users\\dav11274\\Desktop\\github\\Top-20-Python\\Exercises\\Chapter06\\Tutorial_06_02.gdb\\case_open_time_by_neighborhood'